In [10]:
import pandas as pd
# pd.options.display.max_rows = 20
pd.set_option('display.max_colwidth', None)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import copy

In [2]:
import pandas as pd

chunk_size = 20000  # Adjust the chunk size as needed
raw_chunks = pd.read_csv('animelist.csv', chunksize=chunk_size)
# length = sum([1 for _ in chunks])
# print(f"Length: {length}\nitems: < {length * chunk_size}")

In [4]:
chunks = raw_chunks

chunks = []
for chunk in raw_chunks:
    chunks.append(chunk.copy())

colValues = animeDFObj["columns"].copy()
for col in animeDFObj["columnsList"]:
    colValues[col] = 0

animeDF[animeDF["MAL_ID"] == 1 ][animeDFObj["columnsList"]]

In [5]:
animeDF = pd.read_csv('anime.csv')
columns = ["Genres"]
animeDFObj = {}
for chunkColumn in columns:
    animeDFObj["length"] = 0
    animeDFObj["columns"] = {}
    animeDFObj["columnsList"] = ['action', 'adventure', 'comedy', 'drama', 'sci-fi', 'space', 'mystery', 'shounen', 'police', 'supernatural', 'magic', 'fantasy', 'sports', 'josei', 'romance', 'slice of life', 'cars', 'seinen', 'horror', 'psychological', 'thriller', 'super power', 'martial arts', 'school', 'ecchi', 'vampire', 'military', 'historical', 'dementia', 'mecha', 'demons', 'samurai', 'game', 'shoujo', 'harem', 'music', 'shoujo ai', 'shounen ai', 'kids', 'hentai', 'parody', 'yuri', 'yaoi']

    def Genres(row, animeDFObj, isAdd):
        text = row[chunkColumn]
        # print(text)
        text = text.split(",")
        if isAdd:
            animeDFObj["length"] += len(text)
        if isAdd:
            for column in text:
                column = column.lower().strip()
                if column == "unknown": 
                    continue
                # print(animeDFObj["columns"][column])
                if column in animeDFObj["columns"].keys():
                    animeDFObj["columns"][column] +=  1
                else:
                    animeDFObj["columns"][column] = 1
        else:
            for column in text:    
                column = column.lower().strip()
                if column == "unknown": 
                    continue
                row[column] = round(animeDFObj["columns"][column] / animeDFObj["length"],3)
        return row

    animeDF.apply(lambda x: Genres(x, animeDFObj, True),axis = 1)

    animeDFObj["columnsList"] = animeDFObj["columns"].keys()

    for column in animeDFObj["columnsList"]:
        if column == "unknown": 
            continue
        animeDF[column] = np.zeros((animeDF.shape[0], 1), dtype=int)


    animeDF = animeDF.apply(lambda x: Genres(x,animeDFObj,False),axis = 1)

In [6]:
watching_status = pd.read_csv('watching_status.csv')
watching_statusObj = {}
for i in watching_status[["status"," description"]].values:
    # print(i)
    watching_statusObj[i[0]] = i[1]
print(watching_statusObj)

{1: 'Currently Watching', 2: 'Completed', 3: 'On Hold', 4: 'Dropped', 6: 'Plan to Watch'}


In [11]:
chunks = copy.copy(raw_chunks)
for chunk in chunks:
    print(chunk.shape)
  
    chunk = pd.concat([chunk, pd.DataFrame(np.zeros((chunk.shape[0],len(animeDFObj["columnsList"])), dtype=int),columns = animeDFObj["columnsList"])], axis=1)
    chunk["watching_status"] = chunk["watching_status"].apply(lambda x : watching_statusObj[x])
    chunk = pd.get_dummies(chunk, columns=['watching_status'])
    
    # print(chunk)
    for userid,user in chunk.groupby("user_id"):
        columnsList = animeDFObj["columnsList"]
        selected_data = animeDF.loc[animeDF["MAL_ID"] == user["anime_id"].values[0], columnsList]
        print(user.columns)
        for index in range(0,user.shape[0]):
            #user.iloc[0][columnsList] = selected_data.values[0]
        print(selected_data)
        break;
    break;

(20000, 5)
Index(['user_id', 'anime_id', 'rating', 'watched_episodes', 'action',
       'adventure', 'comedy', 'drama', 'sci-fi', 'space', 'mystery', 'shounen',
       'police', 'supernatural', 'magic', 'fantasy', 'sports', 'josei',
       'romance', 'slice of life', 'cars', 'seinen', 'horror', 'psychological',
       'thriller', 'super power', 'martial arts', 'school', 'ecchi', 'vampire',
       'military', 'historical', 'dementia', 'mecha', 'demons', 'samurai',
       'game', 'shoujo', 'harem', 'music', 'shoujo ai', 'shounen ai', 'kids',
       'hentai', 'parody', 'yuri', 'yaoi', 'watching_status_Completed',
       'watching_status_Currently Watching', 'watching_status_Dropped',
       'watching_status_On Hold', 'watching_status_Plan to Watch'],
      dtype='object')
    action  adventure  comedy  drama  sci-fi  space  mystery  shounen  police  \
48   0.077      0.059     0.0    0.0     0.0    0.0      0.0      0.0     0.0   

    supernatural  ...  shoujo  harem  music  shoujo ai  s

In [36]:
user.iloc[0][columnsList] = selected_data.values[0]

KeyError: "['action' 'adventure' 'comedy' 'drama' 'sci-fi' 'space' 'mystery'\n 'shounen' 'police' 'supernatural' 'magic' 'fantasy' 'sports' 'josei'\n 'romance' 'slice of life' 'cars' 'seinen' 'horror' 'psychological'\n 'thriller' 'super power' 'martial arts' 'school' 'ecchi' 'vampire'\n 'military' 'historical' 'dementia' 'mecha' 'demons' 'samurai' 'game'\n 'shoujo' 'harem' 'music' 'shoujo ai' 'shounen ai' 'kids' 'hentai'\n 'parody' 'yuri' 'yaoi'] not in index"

In [38]:
user

,user_id,anime_id,rating,watching_status,watched_episodes
120000,409,457,10,2,26
120001,409,30,7,2,26
120002,409,564,9,2,2
120003,409,208,10,2,3
120004,409,209,8,2,26
...,...,...,...,...,...
120090,409,1603,0,6,1
120091,409,95,0,6,2
120092,409,2840,0,6,0
120093,409,890,8,6,3


In [8]:
raw_chunk["watching_status"] = raw_chunk["watching_status"].apply(lambda x : watching_statusObj[x])

In [9]:
raw_chunk["watching_status"].value_counts()

Completed             11131
Plan to Watch          5979
Currently Watching     1484
Dropped                 800
On Hold                 606
Name: watching_status, dtype: int64

In [10]:
chunk["user_id"].value_counts()

60    1722
17    1188
42    1126
19     967
47     862
      ... 
35      13
43       9
24       7
9        3
26       1
Name: user_id, Length: 71, dtype: int64